In [1]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

from pathlib import Path
import os
import sys
from datetime import datetime, date


In [10]:
result = Path('result/')
report = Path('report/')
instances = Path('instances/csifa')

In [11]:
file_name = sys.argv[1]

In [20]:
def read_instance(file_name):
	
	arq = open(file_name)
	
	N = int(arq.readline())
	
	PR  = [0]*N
	PP  = [0]*N

	FR = [float(arq.readline())]*N
	FP = [float(arq.readline())]*N

	HR = [float(arq.readline())]*N
	HP = [float(arq.readline())]*N

	D = [int(i) for i in arq.readline().split()]
	
	R = [int(i) for i in arq.readline().split()]

	C = float(arq.readline().rstrip('\n'))
	
	return N, PP, PR, FP, FR, HR, HP, D, R, C

In [21]:
MAX_CPU_TIME = 3600.0
EPSILON = 0.000001

def clsr_std_mip(N, PP, PR, FP, FR, HR, HP, D, R, SD, SR, C):
	try:

		# Create a new model
		model = gp.Model("clsr_std_mip")

		# Create variables
		xp = model.addVars(list(range(N)), lb=0.0, ub=float('inf'), vtype=GRB.CONTINUOUS, name="xp")
		yp = model.addVars(list(range(N)), lb=0.0, ub=1.0, vtype=GRB.BINARY, name="yp")
		sp = model.addVars(list(range(N)), lb=0.0, ub=float('inf'), vtype=GRB.CONTINUOUS, name="sp")
		xr = model.addVars(list(range(N)), lb=0.0, ub=float('inf'), vtype=GRB.CONTINUOUS, name="xr")
		yr = model.addVars(list(range(N)), lb=0.0, ub=1.0, vtype=GRB.BINARY, name="yr")
		sr = model.addVars(list(range(N)), lb=0.0, ub=float('inf'), vtype=GRB.CONTINUOUS, name="sr")
		
		model.update()

		## Set objective
		model.setObjective(gp.quicksum(
			PP[i]*xp[i] + sp[i]*HP[i] + 
			xr[i]*PR[i] + sr[i]*HR[i] + 
			yp[i]*FP[i] + yr[i]*FR[i] for i in range(N)), sense = GRB.MINIMIZE)

		## Add constraints
		model.addConstr(xp[0] + xr[0] - sp[0] == D[0])

		model.addConstrs(sp[i-1] + xp[i] + xr[i] - sp[i] == D[i] for i in range(N) if i > 0 )
		
		model.addConstr(R[0] - xr[0] - sr[0] == 0)
		
		model.addConstrs(sr[i-1] + R[i] - xr[i] - sr[i] == 0 for i in range(N) if i > 0)
		
		model.addConstrs(xp[i] - yp[i]*min(SD[i][N-1],C) <= 0 for i in range(N))
		
		model.addConstrs(xr[i] - yr[i]*min(SR[0][i],SD[i][N-1],C) <= 0 for i in range(N))
		
		model.addConstrs(xp[i] + xr[i] <= C for i in range(N))
	    
		# export .lp
		#model.write(file_name+"_model.lp")

		# Parameters 
		model.setParam(GRB.Param.TimeLimit, MAX_CPU_TIME)
		model.setParam(GRB.Param.MIPGap, EPSILON)
		model.setParam(GRB.Param.Threads, 1)
		#model.setParam(GRB.Param.Cuts, -1)
		#model.setParam(GRB.Param.Presolve, -1)

		# Optimize model
		model.optimize()
		
		tmp = 0
		if model.status == GRB.OPTIMAL:
			tmp = 1

		xp_sol = [xp[i].X for i in range(N)]
		xr_sol = [xr[i].X for i in range(N)]
		sp_sol = [sp[i].X for i in range(N)]
		sr_sol = [sr[i].X for i in range(N)]
		yp_sol = [yp[i].X for i in range(N)]
		yr_sol = [yr[i].X for i in range(N)]

		print('Obj: %g' % model.ObjVal)

	except gp.GurobiError as e:
		print('Error code ' + str(e.errno) + ': ' + str(e))

	return model.ObjVal,model.ObjBound,model.MIPGap,model.Runtime,model.NodeCount,tmp, \
				xp_sol,xr_sol,sp_sol,sr_sol, yp_sol,yr_sol


In [24]:
def main():
	
	file_name = f"c52_1.txt"

	N, PP, PR, FP, FR, HR, HP, D, R, C = read_instance(os.path.join(instances,file_name))

	xp_sol = [0]*N
	xr_sol = [0]*N
	sp_sol = [0]*N
	sr_sol = [0]*N
	yp_sol = [0]*N
	yr_sol = [0]*N

	SD = (np.zeros((N,N))).tolist()
	SR = (np.zeros((N,N))).tolist()
	for  i in range(N):
		SD[i][i] = D[i]
		SR[i][i] = R[i]
		for j in range(i+1, N):
			SD[i][j] = SD[i][j-1] + D[j]
			SR[i][j] = SR[i][j-1] + R[j]

	obj, bestbound, gap, temp, numnode, tmp, xp_sol, xr_sol, sp_sol, sr_sol, yp_sol, yr_sol = clsr_std_mip(N, PP, PR, FP, FR, HR, HP, D, R, SD,SR,C)
	
	arquivo = open(os.path.join(result,'clsr_std_mip.txt'),'a')
	
	arquivo.write(file_name+';'+str(round(obj,3))+';'+str(round(bestbound,3))+\
					';'+str(round(gap,3))+';'+str(round(temp,3))+';'+str(round(numnode,3))+\
					';'+str(round(tmp,3))+
					'\n')

	arquivo.close()

	sol_instance = pd.DataFrame()
	sol_instance['xp_sol'] = pd.Series(xp_sol)
	sol_instance['xr_sol'] = pd.Series(xr_sol)
	sol_instance['sp_sol'] = pd.Series(sp_sol)
	sol_instance['sr_sol'] = pd.Series(sr_sol)
	sol_instance['yp_sol'] = pd.Series(yp_sol)
	sol_instance['yr_sol'] = pd.Series(yr_sol)

	sol_instance.to_csv(os.path.join(result,'sol_instance_'+file_name),sep=';',index=False)

In [25]:
if __name__== "__main__" :
	main()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-15


Set parameter TimeLimit to value 3600
Set parameter MIPGap to value 1e-06
Set parameter Threads to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 1 threads

Optimize a model with 260 rows, 312 columns and 674 nonzeros
Model fingerprint: 0x6f2e1d9f
Variable types: 208 continuous, 104 integer (104 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [2e-01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 2e+02]
Found heuristic solution: objective 12770.800000
Presolve removed 5 rows and 5 columns
Presolve time: 0.01s
Presolved: 255 rows, 307 columns, 667 nonzeros
Variable types: 204 continuous, 103 integer (103 binary)

Root relaxation: objective 6.418663e+03, 225 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bou